In [1]:
import csv
import numpy as np
from sentence_transformers import SentenceTransformer

def main():
    input_csv = "日语,英语,汉语的对应翻译列表.csv"            # 入力CSVファイル
    output_csv = "世界语词根的平均语义向量(average_embeddings).csv"   # 出力先のCSVファイル

    # 多言語 SentenceTransformer (LaBSE) をロード
    model = SentenceTransformer('sentence-transformers/LaBSE')

    with open(input_csv, mode='r', encoding='utf-8') as f_in, \
         open(output_csv, mode='w', newline='', encoding='utf-8') as f_out:

        reader = csv.reader(f_in)
        writer = csv.writer(f_out)

        line_count = 0
        for row in reader:
            # CSVの行が空の場合はスキップ
            if not row:
                continue

            # 3列 (ja, en, zh) を取り出し。列が足りない場合は空文字にする。
            ja = row[0].strip() if len(row) > 0 else ""
            en = row[1].strip() if len(row) > 1 else ""
            zh = row[2].strip() if len(row) > 2 else ""

            # 欠損していない言語だけをテキストとしてリストに追加
            texts = []
            if ja:
                texts.append(ja)
            if en:
                texts.append(en)
            if zh:
                texts.append(zh)

            if not texts:
                # すべて空欄(日本語・英語・中国語が欠損)の場合はゼロベクトルを出力
                zero_vec = np.zeros(768, dtype=float)
                writer.writerow(zero_vec.tolist())
            else:
                # エンコード(複数テキスト -> (#texts, 768))
                embeddings = model.encode(texts, convert_to_numpy=True)
                # 平均ベクトル (shape: (768,))
                avg_embedding = np.mean(embeddings, axis=0)
                # CSVに書き込み
                writer.writerow(avg_embedding.tolist())

            line_count += 1
            if line_count % 100 == 0:
                print(f"{line_count} 行を処理しました。")

    print(f"処理完了: 合計 {line_count} 行。出力: {output_csv}")

if __name__ == "__main__":
    main()


/home/yamada/anaconda3/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2025-03-01 20:13:49.170618: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-01 20:13:49.206894: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-01 20:13:50.104520: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


100 行を処理しました。
200 行を処理しました。
300 行を処理しました。
400 行を処理しました。
500 行を処理しました。
600 行を処理しました。
700 行を処理しました。
800 行を処理しました。
900 行を処理しました。
1000 行を処理しました。
1100 行を処理しました。
1200 行を処理しました。
1300 行を処理しました。
1400 行を処理しました。
1500 行を処理しました。
1600 行を処理しました。
1700 行を処理しました。
1800 行を処理しました。
1900 行を処理しました。
2000 行を処理しました。
2100 行を処理しました。
2200 行を処理しました。
2300 行を処理しました。
2400 行を処理しました。
2500 行を処理しました。
2600 行を処理しました。
2700 行を処理しました。
2800 行を処理しました。
2900 行を処理しました。
3000 行を処理しました。
3100 行を処理しました。
3200 行を処理しました。
3300 行を処理しました。
3400 行を処理しました。
3500 行を処理しました。
3600 行を処理しました。
3700 行を処理しました。
3800 行を処理しました。
3900 行を処理しました。
4000 行を処理しました。
4100 行を処理しました。
4200 行を処理しました。
4300 行を処理しました。
4400 行を処理しました。
4500 行を処理しました。
4600 行を処理しました。
4700 行を処理しました。
処理完了: 合計 4736 行。出力: 世界语词根的平均语义向量(average_embeddings).csv


In [2]:
import csv
import numpy as np
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, leaves_list

def main():
    # 前段で作成したファイル（平均ベクトル）
    embeddings_csv = "世界语词根的平均语义向量(average_embeddings).csv"
    # 元のテキストデータ
    original_csv = "世界语词根列表(4736个).csv"
    # 並べ替え後のテキストを保存する先
    reordered_csv = "基于语义向量进行层次聚类的世界语词根列表.csv"

    # 1) 埋め込みベクトルの読み込み
    embeddings = []
    with open(embeddings_csv, mode='r', encoding='utf-8') as f_emb:
        reader = csv.reader(f_emb)
        for row in reader:
            vec = list(map(float, row))  # row を float に変換
            embeddings.append(vec)
    embeddings = np.array(embeddings)  # shape = (N, 768)
    n_rows = embeddings.shape[0]
    print(f"読み込んだベクトルの行数: {n_rows}, 次元: {embeddings.shape[1]}")

    # 2) 元のテキストを行単位で読み込み
    original_lines = []
    with open(original_csv, mode='r', encoding='utf-8') as f_in:
        reader = csv.reader(f_in)
        for row in reader:
            original_lines.append(row)

    # 行数チェック
    if len(original_lines) != n_rows:
        print("警告: input_data.csv と average_embeddings.csv の行数が一致しません。")
        print(f"  テキスト行数: {len(original_lines)}, ベクトル行数: {n_rows}")
        # 必要に応じて処理を終了させるか、短い方に合わせるなどの対応が必要

    # 3) 距離行列を計算（コサイン距離）
    print("行間のコサイン距離を計算しています...")
    dist_array = pdist(embeddings, metric='cosine')

    # 4) 階層クラスタリング
    print("階層クラスタリングを実行しています...")
    # complete法（完全連結法）例。'average', 'single' なども使用可。
    Z = linkage(dist_array, method='complete')  

    # 5) デンドログラムの葉順を取得
    print("デンドログラム葉順を取得...")
    leaf_order = leaves_list(Z)  # shape (N,)

    # ここで leaf_order をファイルに書き出す (テキストファイル)
    print("leaf_order.txt に葉順を保存しています...")
    np.savetxt("leaf_order.txt", leaf_order, fmt='%d')

    # 6) 得られた順序に従って original_lines を並べ替え
    reordered_lines = [original_lines[idx] for idx in leaf_order]

    # 7) 並べ替え後の結果を出力
    print(f"{reordered_csv} に並べ替え結果を出力します...")
    with open(reordered_csv, mode='w', newline='', encoding='utf-8') as f_out:
        writer = csv.writer(f_out)
        for row in reordered_lines:
            writer.writerow(row)

    print("完了しました！")

if __name__ == "__main__":
    main()


読み込んだベクトルの行数: 4736, 次元: 768
行間のコサイン距離を計算しています...
階層クラスタリングを実行しています...
デンドログラム葉順を取得...
leaf_order.txt に葉順を保存しています...
基于语义向量进行层次聚类的世界语词根列表.csv に並べ替え結果を出力します...
完了しました！


In [2]:
import csv
import numpy as np
def reorder_csv(input_csv, output_csv, leaf_order_file="leaf_order.txt"):
    # 並び順を読み込み
    leaf_order = np.loadtxt(leaf_order_file, dtype=int)

    # 入力CSVを行単位で読み込み
    lines = []
    with open(input_csv, mode='r', encoding='utf-8') as f_in:
        reader = csv.reader(f_in)
        for row in reader:
            lines.append(row)

    # 行数が leaf_order と一致するか確認
    n_lines = len(lines)
    if n_lines != len(leaf_order):
        print("行数が一致しません。並び替え不可。")
        return

    # 並び替えを適用
    reordered_lines = [lines[idx] for idx in leaf_order]

    # 結果を書き出し
    with open(output_csv, mode='w', newline='', encoding='utf-8') as f_out:
        writer = csv.writer(f_out)
        writer.writerows(reordered_lines)

    print(f"完了: {input_csv} を {output_csv} に並び替え出力しました。")

# 使い方:
reorder_csv("用于交由生成AI决定汉字分配的世界语词根及其日语,英语,汉语对应翻译列表.csv", "基于世界语词根的平均语义向量进行层次聚类排序后的,用于交由生成AI决定汉字分配的世界语词根及其日语,英语,汉语对应翻译列表.csv", leaf_order_file="leaf_order.txt")


完了: 用于交由生成AI决定汉字分配的世界语词根及其日语,英语,汉语对应翻译列表.csv を 基于世界语词根的平均语义向量进行层次聚类排序后的,用于交由生成AI决定汉字分配的世界语词根及其日语,英语,汉语对应翻译列表.csv に並び替え出力しました。
